In [1]:
(require '[clojupyter.misc.helper :as helper])

nil

In [2]:
(->> '[[cnuernber/libpython-clj "1.2"]
       [panthera "0.1-alpha.16"]]
     (map helper/add-dependencies)
     doall)

:done

:done

In [3]:
(ns panthera-test
  (:require
   [libpython-clj.python :as py]
   [panthera.panthera :as pt]
   [panthera.pandas.utils :as u]
   [clojure.pprint :refer [pprint]]
   [clojure.repl :refer [doc]]))

nil

In [4]:
(py/initialize!)

:ok

In [5]:
(defn py-get-in
  "A similar to `get-in` implementation for Python modules,
  classes and functions."
  [py-module v]
  (let [mods (drop-last v)]
    ((apply comp
            (reverse
             (map (fn [x] #(py/get-attr % x)) mods))) py-module)))

(defn module
  [py-module]
  (fn [x]
      (fn
        ([]
         (if (seqable? x)
           (let [ks (map u/memo-key-converter x)]
             (py/get-attr (py-get-in py-module ks) (last ks)))
           (py/get-attr py-module (u/memo-key-converter x))))
        ([attrs]
         (if (seqable? x)
           (let [ks    (map u/memo-key-converter x)]
             (py/call-attr-kw (py-get-in py-module ks) (last ks)
                              (vec (:args attrs))
                              (u/keys->pyargs (dissoc attrs :args))))
           (py/call-attr-kw py-module (u/memo-key-converter x)
                            (vec (:args attrs))
                            (u/keys->pyargs (dissoc attrs :args))))))))

#'panthera-test/module

In [6]:
(defonce sk (py/import-module "sklearn"))

; sklearn architecture is very convoluted, modules aren't loaded by default
; but only by explicit import. Setting the var below bypasses this behaviour
;; (py/set-attr! sk "__SKLEARN_SETUP__" true)

(defn sklearn
  ([k] ((module sk) k))
  ([k args] (((module sk) k) args)))

#'panthera-test/sklearn

In [8]:
(def test_set (sklearn [:datasets :fetch-20newsgroups]
                       {:subset "test", :shuffle true}))

Syntax error (ExceptionInfo) compiling at (REPL:1:15).
AttributeError: module 'sklearn' has no attribute 'datasets'



class clojure.lang.Compiler$CompilerException: 